In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
from scipy import stats
from sklearn.preprocessing import StandardScaler
import nltk
import seaborn as sns
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_moons
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter


import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

%matplotlib inline

In [2]:
# Check how the dataset looks like
df = pd.read_csv("Cluster_Hackathon_dump.csv")
df.head()

,App_ID,amount,balance,bank_statement_id,Transaction_Date,Transaction_Details,lender,Bank,File_ID,Has_Password,IFSC,Bank Statements__statement_from,Bank Statements__statement_to
0,7663,-240.0,9769.97,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-10T05:30:00+05:30,APY5759796_102018_501203751878_INSTALLME,NaN,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30
1,7663,-1800.0,22586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-29T05:30:00+05:30,UPI-50100188029243-SRINATHAR56@OKHDFCBANK-PAY-...,NaN,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30
2,7663,-2000.0,20586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,NaN,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30
3,7663,113.0,20699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,CREDIT INTEREST CAPITALISED,NaN,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30
4,7663,-10000.0,10699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-01T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,NaN,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30


In [3]:
#Checking the null values
df.isnull().sum()

App_ID                                  0
amount                                  0
balance                                 0
bank_statement_id                       0
Transaction_Date                        0
Transaction_Details                     0
lender                             196178
Bank                                    0
File_ID                                 0
Has_Password                            0
IFSC                                21355
Bank Statements__statement_from         0
Bank Statements__statement_to           0
dtype: int64

We have two features where we are able to see some null values. First is IFSC , second is lender. To simplify,For IFSC we will drop the rows with null values. For lender, we will drop the whole feature as we are not getting enough information from the same

In [4]:
df = df[df['IFSC'].notna()]
df = df.drop(['lender'],axis=1)

In [5]:
df.isnull().sum()

App_ID                             0
amount                             0
balance                            0
bank_statement_id                  0
Transaction_Date                   0
Transaction_Details                0
Bank                               0
File_ID                            0
Has_Password                       0
IFSC                               0
Bank Statements__statement_from    0
Bank Statements__statement_to      0
dtype: int64

In [6]:
# Looking at the data I feel Transaction_Date is of no reelvance ,so trying to make something out of it for EDA
df['Transaction_Date_Short'] = df['Transaction_Date'].str[:10]
df['Transaction_Date_Short']= pd.to_datetime(df['Transaction_Date_Short'], format ="%Y-%m-%d")
df['Transaction_Date_Day'] = df['Transaction_Date_Short'].dt.day_name()

In [7]:
df.head()

,App_ID,amount,balance,bank_statement_id,Transaction_Date,Transaction_Details,Bank,File_ID,Has_Password,IFSC,Bank Statements__statement_from,Bank Statements__statement_to,Transaction_Date_Short,Transaction_Date_Day
0,7663,-240.0,9769.97,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-10T05:30:00+05:30,APY5759796_102018_501203751878_INSTALLME,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-10-10,Wednesday
1,7663,-1800.0,22586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-29T05:30:00+05:30,UPI-50100188029243-SRINATHAR56@OKHDFCBANK-PAY-...,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-29,Saturday
2,7663,-2000.0,20586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-30,Sunday
3,7663,113.0,20699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,CREDIT INTEREST CAPITALISED,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-30,Sunday
4,7663,-10000.0,10699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-01T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-10-01,Monday


In [8]:
# I see some irregularities in FIle Id which I am removing 
df = df[df.File_ID != 'kotak']
df.shape

(178645, 14)

In [12]:
import re
def cleanDetails(detailsText):
    detailsText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', detailsText)  # remove punctuations
    detailsText = re.sub('\s+', ' ', detailsText)  # remove extra whitespace
    return detailsText
    
df['Transaction_Details_Cleaned'] = df.Transaction_Details.apply(lambda x: cleanDetails(x))

In [13]:
df.head()

,App_ID,amount,balance,bank_statement_id,Transaction_Date,Transaction_Details,Bank,File_ID,Has_Password,IFSC,Bank Statements__statement_from,Bank Statements__statement_to,Transaction_Date_Short,Transaction_Date_Day,Transaction_Details_Cleaned
0,7663,-240.0,9769.97,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-10T05:30:00+05:30,APY5759796_102018_501203751878_INSTALLME,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-10-10,Wednesday,APY5759796 102018 501203751878 INSTALLME
1,7663,-1800.0,22586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-29T05:30:00+05:30,UPI-50100188029243-SRINATHAR56@OKHDFCBANK-PAY-...,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-29,Saturday,UPI 50100188029243 SRINATHAR56 OKHDFCBANK PAY ...
2,7663,-2000.0,20586.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-30,Sunday,NWD 652166XXXXXX5428 42390431 MALUR
3,7663,113.0,20699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-09-30T05:30:00+05:30,CREDIT INTEREST CAPITALISED,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-09-30,Sunday,CREDIT INTEREST CAPITALISED
4,7663,-10000.0,10699.09,dc0b54bf-4fa8-4997-891c-8e85a67c1c06,2018-10-01T05:30:00+05:30,NWD-652166XXXXXX5428-42390431-MALUR,hdfc,199122,True,HDFC0009061,2018-09-29T05:30:00+05:30,2018-10-28T05:30:00+05:30,2018-10-01,Monday,NWD 652166XXXXXX5428 42390431 MALUR


In [15]:
# Gives the most common words with count
oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])
totalWords =[]
Sentences = df['Transaction_Details_Cleaned'].values
cleanedSentences = ""
for i in range(0,178644):
    cleanedText = cleanDetails(Sentences[i])
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word.lower())
    
wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)
print(mostcommon)


[('upi', 70574), ('imps', 18148), ('pos', 17333), ('paytm', 11447), ('debit', 9696), ('cash', 9491), ('payment', 9135), ('na', 8389), ('mmt', 7410), ('ybl', 7058), ('18', 6984), ('hdfc', 6961), ('bank', 6611), ('p2a', 6358), ('from', 6197), ('paymentfro', 6157), ('dt', 6057), ('ref', 5957), ('mb', 5467), ('value', 5188), ('atm', 4941), ('pay', 4923), ('pcd', 4860), ('2018', 4711), ('wdl', 4519), ('0', 4302), ('bil', 4138), ('nwd', 4049), ('ltd', 4023), ('19', 3855), ('2019', 3750), ('phonepe', 3562), ('neft', 3308), ('10', 3262), ('openingbalance', 3242), ('atl', 3135), ('01', 3128), ('valuedate', 2782), ('nfs', 2769), ('chrg', 2688), ('atw', 2685), ('icici', 2666), ('money', 2586), ('05', 2549), ('07', 2537), ('p', 2514), ('09', 2441), ('08', 2404), ('17', 2391), ('06', 2385)]
